In [1]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode, modify by wmathor
  Reference : https://github.com/jadore801120/attention-is-all-you-need-pytorch
         https://github.com/JayParks/transformer, https://github.com/dhlee347/pytorchic-bert
'''
import re
import math
import torch
import numpy as np
from random import *
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

text = (
    'Hello, how are you? I am Romeo.\n' # R
    'Hello, Romeo My name is Juliet. Nice to meet you.\n' # J
    'Nice meet you too. How are you today?\n' # R
    'Great. My baseball team won the competition.\n' # J
    'Oh Congratulations, Juliet\n' # R
    'Thank you Romeo\n' # J
    'Where are you going today?\n' # R
    'I am going shopping. What about you?\n' # J
    'I am going to visit my grandmother. she is not very well' # R
)
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n') # filter '.', ',', '?', '!'
word_list = list(set(" ".join(sentences).split())) # ['hello', 'how', 'are', 'you',...]
word2idx = {'[PAD]' : 0, '[CLS]' : 1, '[SEP]' : 2, '[MASK]' : 3}
for i, w in enumerate(word_list):
    word2idx[w] = i + 4
idx2word = {i: w for i, w in enumerate(word2idx)}
vocab_size = len(word2idx)

token_list = list()
for sentence in sentences:
    arr = [word2idx[s] for s in sentence.split()]
    token_list.append(arr)


In [2]:
# BERT Parameters
maxlen = 30
batch_size = 6
max_pred = 5 # max tokens of prediction
n_layers = 6
n_heads = 12
d_model = 768
d_ff = 768*4 # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

In [3]:
# sample IsNext and NotNext to be same in small batch size
def make_data():
    batch = []
    positive = negative = 0
    while positive != batch_size/2 or negative != batch_size/2:
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences)) # sample random index in sentences
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]
        input_ids = [word2idx['[CLS]']] + tokens_a + [word2idx['[SEP]']] + tokens_b + [word2idx['[SEP]']]
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        # MASK LM
        n_pred =  min(max_pred, max(1, int(len(input_ids) * 0.15))) # 15 % of tokens in one sentence
        cand_maked_pos = [i for i, token in enumerate(input_ids)
                          if token != word2idx['[CLS]'] and token != word2idx['[SEP]']] # candidate masked position
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            if random() < 0.8:  # 80%
                input_ids[pos] = word2idx['[MASK]'] # make mask
            elif random() > 0.9:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                while index < 4: # can't involve 'CLS', 'SEP', 'PAD'
                  index = randint(0, vocab_size - 1)
                input_ids[pos] = index # replace

        # Zero Paddings
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # Zero Padding (100% - 15%) tokens
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
    return batch
# Proprecessing Finished

In [4]:
batch = make_data()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = zip(*batch)
input_ids, segment_ids, masked_tokens, masked_pos, isNext = \
    torch.LongTensor(input_ids),  torch.LongTensor(segment_ids), torch.LongTensor(masked_tokens),\
    torch.LongTensor(masked_pos), torch.LongTensor(isNext)

class MyDataSet(Data.Dataset):
  def __init__(self, input_ids, segment_ids, masked_tokens, masked_pos, isNext):
    self.input_ids = input_ids
    self.segment_ids = segment_ids
    self.masked_tokens = masked_tokens
    self.masked_pos = masked_pos
    self.isNext = isNext
  
  def __len__(self):
    return len(self.input_ids)
  
  def __getitem__(self, idx):
    return self.input_ids[idx], self.segment_ids[idx], self.masked_tokens[idx], self.masked_pos[idx], self.isNext[idx]

loader = Data.DataLoader(MyDataSet(input_ids, segment_ids, masked_tokens, masked_pos, isNext), batch_size, True)

In [5]:
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, seq_len = seq_q.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_q.data.eq(0).unsqueeze(1)  # [batch_size, 1, seq_len]
    return pad_attn_mask.expand(batch_size, seq_len, seq_len)  # [batch_size, seq_len, seq_len]

def gelu(x):
    """
      Implementation of the gelu activation function.
      For information: OpenAI GPT's gelu is slightly different (and gives slightly different results):
      0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
      Also see https://arxiv.org/abs/1606.08415
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # [seq_len] -> [batch_size, seq_len]
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size, n_heads, seq_len, seq_len]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size, seq_len, d_model], k: [batch_size, seq_len, d_model], v: [batch_size, seq_len, d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size, n_heads, seq_len, d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size, n_heads, seq_len, d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size, n_heads, seq_len, d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]

        # context: [batch_size, n_heads, seq_len, d_v], attn: [batch_size, n_heads, seq_len, seq_len]
        context = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size, seq_len, n_heads, d_v]
        output = nn.Linear(n_heads * d_v, d_model)(context)
        return nn.LayerNorm(d_model)(output + residual) # output: [batch_size, seq_len, d_model]

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_ff) -> (batch_size, seq_len, d_model)
        return self.fc2(gelu(self.fc1(x)))

class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, seq_len, d_model]
        return enc_outputs

class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.Dropout(0.5),
            nn.Tanh(),
        )
        self.classifier = nn.Linear(d_model, 2)
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu
        # fc2 is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        self.fc2 = nn.Linear(d_model, vocab_size, bias=False)
        self.fc2.weight = embed_weight

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids) # [bach_size, seq_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids) # [batch_size, maxlen, maxlen]
        for layer in self.layers:
            # output: [batch_size, max_len, d_model]
            output = layer(output, enc_self_attn_mask)
        # it will be decided by first token(CLS)
        h_pooled = self.fc(output[:, 0]) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2] predict isNext

        masked_pos = masked_pos[:, :, None].expand(-1, -1, d_model) # [batch_size, max_pred, d_model]
        print(output)
        print(masked_pos)
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        print(h_masked)
        h_masked = self.activ2(self.linear(h_masked)) # [batch_size, max_pred, d_model]
        logits_lm = self.fc2(h_masked) # [batch_size, max_pred, vocab_size]
        return logits_lm, logits_clsf
model = BERT()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters(), lr=0.001)

In [6]:
for epoch in range(50):
    for input_ids, segment_ids, masked_tokens, masked_pos, isNext in loader:
      logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
      loss_lm = criterion(logits_lm.view(-1, vocab_size), masked_tokens.view(-1)) # for masked LM
      loss_lm = (loss_lm.float()).mean()
      loss_clsf = criterion(logits_clsf, isNext) # for sentence classification
      loss = loss_lm + loss_clsf
      if (epoch + 1) % 10 == 0:
          print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

tensor([[[ 0.0150,  0.0803,  0.1335,  ..., -0.3374, -0.3911, -0.1357],
         [ 0.0618,  0.1217,  0.1166,  ..., -0.3112, -0.4714, -0.1042],
         [ 0.0191, -0.1239,  0.0297,  ..., -0.2181, -0.3603, -0.0241],
         ...,
         [ 0.0772,  0.1451,  0.1039,  ..., -0.3639, -0.3891, -0.1062],
         [ 0.1547,  0.0765,  0.3791,  ..., -0.4754, -0.3614, -0.0632],
         [ 0.1023, -0.1089,  0.2651,  ..., -0.2462, -0.2246, -0.1020]],

        [[-0.0158,  0.0852,  0.1496,  ..., -0.3371, -0.3361, -0.2252],
         [ 0.2166,  0.0245,  0.1520,  ..., -0.4077, -0.3218,  0.0730],
         [-0.0993, -0.0466, -0.0851,  ..., -0.2425, -0.3154, -0.1258],
         ...,
         [ 0.0849,  0.2205,  0.1768,  ..., -0.3987, -0.3296, -0.1214],
         [ 0.0749,  0.1199,  0.3998,  ..., -0.4249, -0.3395, -0.1028],
         [ 0.1194, -0.0553,  0.2519,  ..., -0.2607, -0.2059, -0.1631]],

        [[-0.0107,  0.1061,  0.1554,  ..., -0.3116, -0.3071, -0.1047],
         [-0.0178, -0.0493,  0.1607,  ..., -0

tensor([[[-0.0228,  0.0656,  0.1545,  ..., -0.1720,  0.0957,  0.2239],
         [-0.0444,  0.0155, -0.0922,  ..., -0.1839, -0.0811,  0.1703],
         [-0.1373, -0.0934,  0.1707,  ..., -0.1467,  0.0479,  0.1973],
         ...,
         [-0.0060, -0.0056,  0.1709,  ..., -0.1987, -0.1090,  0.1479],
         [-0.0032, -0.0122,  0.3005,  ..., -0.3724, -0.1475,  0.2039],
         [ 0.0313, -0.0902,  0.1320,  ..., -0.2628, -0.0077,  0.1501]],

        [[-0.0385,  0.1181,  0.1162,  ..., -0.1644,  0.1541,  0.2183],
         [ 0.0928, -0.0966, -0.0299,  ..., -0.5027, -0.0795,  0.2208],
         [-0.4651, -0.0932, -0.0381,  ..., -0.2185,  0.0760,  0.0272],
         ...,
         [ 0.0056,  0.0172,  0.1593,  ..., -0.1823, -0.0318,  0.1085],
         [ 0.0349,  0.0462,  0.2897,  ..., -0.3616, -0.0601,  0.1113],
         [ 0.0192, -0.0361,  0.0916,  ..., -0.2205, -0.0067,  0.1678]],

        [[ 0.0299,  0.0761,  0.1414,  ..., -0.1759,  0.0938,  0.2133],
         [-0.0208,  0.0568,  0.0321,  ..., -0

tensor([[[ 1.0727e-01,  2.0586e-01, -9.5499e-03,  ..., -3.9485e-01,
           4.5807e-02, -1.3133e-01],
         [ 2.1407e-01,  6.1268e-02,  9.9928e-03,  ..., -2.3036e-01,
          -7.8879e-02, -1.4146e-04],
         [ 6.6606e-04,  4.3951e-02, -2.2786e-01,  ..., -1.9354e-01,
          -7.2799e-02, -4.2412e-02],
         ...,
         [ 1.2831e-01,  2.0523e-01, -1.9605e-03,  ..., -2.5680e-01,
          -2.0889e-01,  1.6366e-02],
         [ 1.4610e-01,  1.7800e-01,  2.1594e-01,  ..., -5.1211e-01,
          -1.5636e-01,  1.6795e-01],
         [ 5.2598e-02,  9.4322e-02,  1.4128e-01,  ..., -1.6139e-01,
          -3.5008e-02,  5.0725e-02]],

        [[ 9.8840e-02,  2.1264e-01,  1.6759e-03,  ..., -3.6480e-01,
          -4.7370e-03, -1.7283e-01],
         [ 1.4008e-01,  1.5641e-01, -1.6904e-02,  ..., -2.1028e-01,
          -1.6264e-01, -1.6149e-01],
         [ 2.7039e-01,  6.2256e-02, -4.9685e-02,  ..., -3.0533e-01,
          -9.7589e-02, -4.3638e-02],
         ...,
         [ 9.8611e-02,  2

tensor([[[ 0.1249,  0.2139,  0.0741,  ..., -0.3574, -0.2017, -0.0032],
         [ 0.1047,  0.2048, -0.0505,  ..., -0.3441, -0.0971, -0.0725],
         [ 0.1459,  0.0270,  0.0532,  ..., -0.3920, -0.1251,  0.1291],
         ...,
         [ 0.2061,  0.1866,  0.1231,  ..., -0.4352, -0.1675,  0.1245],
         [ 0.1970,  0.1577,  0.3981,  ..., -0.5813, -0.1088,  0.2193],
         [ 0.1104,  0.1028,  0.0941,  ..., -0.2478, -0.0438,  0.1709]],

        [[ 0.0610,  0.2664,  0.1059,  ..., -0.3174, -0.2440, -0.0053],
         [ 0.0476,  0.0616, -0.0374,  ..., -0.3224, -0.1740,  0.1494],
         [-0.1165,  0.2155,  0.0110,  ..., -0.3376, -0.0808,  0.0198],
         ...,
         [ 0.1682,  0.2178,  0.1478,  ..., -0.4723, -0.1797,  0.1267],
         [ 0.1713,  0.1871,  0.4113,  ..., -0.5911, -0.1432,  0.2329],
         [ 0.0715,  0.1227,  0.1313,  ..., -0.2827, -0.0602,  0.1582]],

        [[ 0.1181,  0.2653,  0.1096,  ..., -0.3692, -0.3136,  0.0072],
         [ 0.1208,  0.1664, -0.0194,  ..., -0

tensor([[[ 1.9250e-01,  3.8072e-01,  1.1791e-01,  ..., -1.0606e-01,
           2.9466e-02,  2.6639e-02],
         [ 1.4175e-01,  3.0009e-01,  4.4127e-02,  ..., -1.9462e-01,
          -1.8417e-01, -8.3322e-02],
         [ 1.4853e-01,  1.2062e-01, -9.0015e-02,  ..., -9.3508e-02,
           2.7332e-02, -3.2882e-01],
         ...,
         [ 2.1124e-01,  1.5213e-01,  2.7659e-02,  ..., -2.2728e-01,
          -8.8903e-02, -1.9994e-01],
         [ 1.5941e-01,  2.2887e-01,  2.1698e-01,  ..., -3.4272e-01,
          -1.4041e-01, -2.5159e-02],
         [ 1.0716e-01,  4.3527e-02,  1.6366e-01,  ..., -6.4023e-02,
          -4.0749e-02, -1.6748e-01]],

        [[ 1.9954e-01,  3.7819e-01,  9.0425e-02,  ..., -1.3831e-01,
           1.5226e-02,  3.8371e-02],
         [ 2.6849e-01,  1.8602e-01,  1.5387e-01,  ..., -2.7568e-01,
          -1.2610e-01, -1.8698e-01],
         [-1.9623e-01,  1.9743e-01,  1.2035e-01,  ..., -2.6188e-01,
           4.1125e-02, -2.4891e-01],
         ...,
         [ 2.1129e-01,  1

...

tensor([[[-1.3971e-02,  1.6734e-01,  1.4760e-01,  ..., -7.9969e-02,
          -7.4909e-02, -1.7170e-01],
         [ 2.6353e-01,  1.0887e-01, -3.0726e-02,  ..., -2.7440e-01,
          -2.7288e-01,  2.9347e-02],
         [-1.9998e-01,  2.0377e-02, -8.3577e-02,  ..., -1.3478e-01,
          -1.2508e-02, -7.6427e-02],
         ...,
         [ 2.1549e-01,  1.3468e-01,  1.7366e-01,  ..., -2.1289e-01,
          -4.3143e-01,  3.1832e-02],
         [ 2.9547e-01,  2.2665e-01,  2.5475e-01,  ..., -2.3428e-01,
          -2.8201e-01,  2.3576e-02],
         [ 2.8110e-01,  5.4864e-02,  1.2143e-01,  ..., -8.0708e-02,
          -2.1251e-01, -1.7581e-01]],

        [[ 7.2166e-03,  1.8540e-01,  8.0365e-02,  ..., -1.2230e-01,
          -7.8814e-02, -1.7042e-01],
         [ 1.8241e-02,  2.3287e-01,  2.6398e-02,  ..., -4.0637e-01,
          -3.4825e-01, -2.5984e-04],
         [-8.0930e-02, -2.5953e-02, -2.0414e-01,  ..., -8.3779e-02,
          -1.4310e-01, -2.0818e-01],
         ...,
         [ 2.5890e-01,  1

tensor([[[-0.0334,  0.2658,  0.1294,  ..., -0.3646, -0.2184, -0.0817],
         [-0.0615,  0.2708,  0.1594,  ..., -0.3895, -0.3595,  0.0938],
         [ 0.0657,  0.0231, -0.0865,  ..., -0.0925, -0.0177,  0.0054],
         ...,
         [ 0.1372,  0.1231,  0.0412,  ..., -0.2142, -0.3604, -0.0227],
         [ 0.1629,  0.2417,  0.2912,  ..., -0.4163, -0.3327,  0.1296],
         [ 0.1305,  0.1231,  0.1652,  ..., -0.1483, -0.3030,  0.0926]],

        [[-0.0621,  0.2669,  0.1046,  ..., -0.3565, -0.2744, -0.0886],
         [ 0.1222, -0.0402,  0.1264,  ..., -0.3270, -0.4386,  0.0772],
         [-0.2476,  0.0599, -0.1901,  ..., -0.2195, -0.1011, -0.1187],
         ...,
         [ 0.1500,  0.0667, -0.0011,  ..., -0.2079, -0.4292, -0.0957],
         [ 0.1160,  0.2441,  0.2561,  ..., -0.4647, -0.4021,  0.0669],
         [ 0.0974,  0.0554,  0.1861,  ..., -0.1453, -0.3533,  0.0670]],

        [[-0.0329,  0.2514,  0.0600,  ..., -0.3669, -0.2726, -0.1273],
         [ 0.1343,  0.1964,  0.0726,  ..., -0

tensor([[[-0.0624,  0.4178,  0.1188,  ..., -0.2217,  0.0522,  0.1910],
         [-0.1654, -0.0031, -0.0533,  ..., -0.1364, -0.0737, -0.0078],
         [-0.3550,  0.1038, -0.0404,  ..., -0.1816, -0.0869,  0.0301],
         ...,
         [ 0.0594,  0.0524,  0.1749,  ..., -0.2240, -0.2220, -0.1007],
         [ 0.1319,  0.0988,  0.3121,  ..., -0.3675, -0.3067,  0.0526],
         [-0.0865, -0.0565,  0.1548,  ..., -0.1663, -0.0310, -0.0647]],

        [[-0.0731,  0.3666,  0.1064,  ..., -0.2150,  0.0760,  0.2232],
         [-0.1503,  0.1640, -0.0235,  ..., -0.2560, -0.2187,  0.1516],
         [-0.2081,  0.0254, -0.0548,  ..., -0.0150, -0.0073,  0.1054],
         ...,
         [ 0.0695,  0.0493,  0.1688,  ..., -0.2627, -0.2151,  0.0200],
         [ 0.1139,  0.0446,  0.2979,  ..., -0.3740, -0.3184,  0.1438],
         [-0.1155, -0.1050,  0.1441,  ..., -0.1836, -0.0154, -0.0152]],

        [[-0.0476,  0.4083,  0.1394,  ..., -0.2065,  0.0818,  0.1897],
         [ 0.0974, -0.0638, -0.0033,  ..., -0

tensor([[[-1.4417e-01,  2.6365e-01,  2.5368e-01,  ..., -2.5475e-01,
          -1.7271e-02,  1.3302e-01],
         [ 4.2612e-02,  1.7545e-01, -9.1141e-02,  ..., -2.4009e-01,
          -1.0801e-01, -6.6152e-02],
         [ 5.1491e-02, -9.8267e-02, -2.8291e-01,  ..., -2.2033e-01,
          -4.8582e-02,  9.2415e-02],
         ...,
         [-8.7097e-02,  1.4182e-01,  7.8921e-02,  ..., -2.3432e-01,
          -2.8608e-01, -2.4909e-02],
         [ 4.6073e-02,  8.5275e-02,  2.2382e-01,  ..., -2.6793e-01,
          -1.6501e-01,  5.8066e-02],
         [-9.0084e-02,  1.4203e-01,  1.4989e-01,  ..., -1.3411e-01,
          -1.0319e-01,  6.1693e-02]],

        [[-1.3082e-01,  2.3478e-01,  2.2200e-01,  ..., -2.4169e-01,
          -2.8670e-02,  5.0762e-02],
         [-2.3651e-02,  2.7595e-02,  7.3399e-02,  ..., -2.6565e-01,
          -3.4617e-01, -4.1058e-03],
         [-2.9819e-01, -1.2732e-01, -1.0045e-01,  ..., -6.3821e-02,
           6.7644e-02,  4.1613e-02],
         ...,
         [-8.4998e-02,  1

tensor([[[-0.0297,  0.1708,  0.3825,  ..., -0.1184,  0.0157, -0.1317],
         [ 0.2721, -0.0909,  0.2505,  ..., -0.2795, -0.1713, -0.0327],
         [-0.2040, -0.1758, -0.0304,  ..., -0.2477,  0.0096, -0.1276],
         ...,
         [ 0.1037,  0.0358,  0.2597,  ..., -0.1078, -0.2125, -0.0411],
         [ 0.1731,  0.0440,  0.4619,  ..., -0.2471, -0.0451,  0.0098],
         [-0.0355, -0.1314,  0.4633,  ...,  0.0693, -0.0266, -0.0248]],

        [[ 0.0190,  0.1686,  0.3185,  ..., -0.1305, -0.0085, -0.0573],
         [ 0.2615, -0.1131,  0.3118,  ..., -0.3475, -0.1727, -0.0536],
         [-0.1379, -0.2416,  0.1280,  ..., -0.2799,  0.0328, -0.0084],
         ...,
         [ 0.2342,  0.0588,  0.1746,  ..., -0.1636, -0.3627, -0.0339],
         [ 0.2835, -0.0010,  0.4205,  ..., -0.2834, -0.0864,  0.0534],
         [ 0.1415, -0.1580,  0.4150,  ..., -0.0342, -0.0767,  0.0498]],

        [[-0.0208,  0.1928,  0.3636,  ..., -0.1637, -0.0315, -0.0530],
         [ 0.0970,  0.0218,  0.2331,  ..., -0

In [7]:
# Predict mask tokens ans isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = batch[1]
print(text)
print('================================')
print([idx2word[w] for w in input_ids if idx2word[w] != '[PAD]'])

logits_lm, logits_clsf = model(torch.LongTensor([input_ids]), \
                 torch.LongTensor([segment_ids]), torch.LongTensor([masked_pos]))
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('masked tokens list : ',[pos for pos in masked_tokens if pos != 0])
print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_clsf else False)

Hello, how are you? I am Romeo.
Hello, Romeo My name is Juliet. Nice to meet you.
Nice meet you too. How are you today?
Great. My baseball team won the competition.
Oh Congratulations, Juliet
Thank you Romeo
Where are you going today?
I am going shopping. What about you?
I am going to visit my grandmother. she is not very well
['[CLS]', 'oh', 'congratulations', 'juliet', '[SEP]', 'hello', 'how', '[MASK]', 'you', 'i', 'am', 'romeo', '[SEP]']
tensor([[[ 0.1184,  0.3888,  0.2697,  ..., -0.3624, -0.3447, -0.0079],
         [ 0.1363,  0.1355,  0.0647,  ..., -0.3423, -0.4019, -0.0362],
         [ 0.0880,  0.0470,  0.0516,  ..., -0.3133, -0.1603,  0.1373],
         ...,
         [ 0.1700,  0.1891,  0.1666,  ..., -0.4446, -0.3746,  0.0226],
         [ 0.2080,  0.2752,  0.3133,  ..., -0.5361, -0.3782,  0.0177],
         [ 0.3132,  0.0837,  0.2141,  ..., -0.2997, -0.3266, -0.0464]]],
       grad_fn=<AddBackward0>)
tensor([[[7, 7, 7,  ..., 7, 7, 7],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [8]:
#应用量化
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

BERT(
  (embedding): Embedding(
    (tok_embed): Embedding(40, 768)
    (pos_embed): Embedding(30, 768)
    (seg_embed): Embedding(2, 768)
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (layers): ModuleList(
    (0): EncoderLayer(
      (enc_self_attn): MultiHeadAttention(
        (W_Q): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
        (W_K): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
        (W_V): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
      )
      (pos_ffn): PoswiseFeedForwardNet(
        (fc1): DynamicQuantizedLinear(in_features=768, out_features=3072, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
        (fc2): DynamicQuantizedLinear(in_features=3072, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
      )
    )
    (1): Enc

In [14]:
#输出对比测试结果
import os
import time

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

def print_time_of_model(model):    
    eval_start_time = time.time()
    model(torch.LongTensor([input_ids]), \
                 torch.LongTensor([segment_ids]), torch.LongTensor([masked_pos]))
    eval_end_time = time.time()
    eval_duration_time = eval_end_time - eval_start_time
    print("Evaluate total time (seconds): {0:.1f}".format(eval_duration_time))

    
print_size_of_model(model)
print_size_of_model(quantized_model)

print_time_of_model(model)
print_time_of_model(quantized_model)

Size (MB): 160.844839
Size (MB): 40.572431
tensor([[[-0.0738,  0.3327,  0.0380,  ..., -0.3048, -0.0289, -0.1779],
         [ 0.1013,  0.3634,  0.0050,  ..., -0.1247, -0.0733, -0.1555],
         [-0.0312,  0.2100, -0.0643,  ..., -0.1972, -0.0326, -0.1339],
         ...,
         [-0.0090,  0.1465,  0.0373,  ..., -0.2756, -0.2431, -0.1753],
         [-0.0873,  0.1474,  0.2928,  ..., -0.3138, -0.1397, -0.0847],
         [ 0.0250,  0.1036,  0.1591,  ..., -0.1132, -0.0359, -0.0897]]],
       grad_fn=<AddBackward0>)
tensor([[[7, 7, 7,  ..., 7, 7, 7],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]])
tensor([[[ 0.0538,  0.1581,  0.0941,  ..., -0.4250,  0.0572,  0.0348],
         [-0.0738,  0.3327,  0.0380,  ..., -0.3048, -0.0289, -0.1779],
         [-0.0738,  0.3327,  0.0380,  ..., -0.3048, -0.0289, -0.1779],
         [-0.0738,  0.3327,  0.0380,  ..., -0.3048, -0.0289, -0.1779],
         [-0.0738,  0.3